In [ ]:
!nvidia-smi

Mon Oct 12 14:51:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !fusermount -u google_drive

In [ ]:
from IPython.display import clear_output

!add-apt-repository -y ppa:alessandro-strada/ppa; 
!apt-get update;
!apt-get install -y google-drive-ocamlfuse;
clear_output()

!mkdir google_drive;
!google-drive-ocamlfuse -headless -label dacon_smiles -id 406775554485-vqr231cgnpofc9mkm7sr0e3uq32emf11.apps.googleusercontent.com -secret iFy1t7pKRjOzBuWHbUB-cM8V;

!sed -i 's/team_drive_id=0AOLSYhuNgxEsUk9PVA/team_drive_id=/' ~/.gdfuse/dacon_smiles/config
!sed -i 's/team_drive_id=/team_drive_id=0AOLSYhuNgxEsUk9PVA/' ~/.gdfuse/dacon_smiles/config
!google-drive-ocamlfuse -label dacon_smiles google_drive/

# !fusermount -u google_drive

# rdkit 2020.03.3 버전 다운로드
!pip install kora -q
import kora.install.rdkit

import os
import os.path as pth

### 저는 코랩에서 구글 드라이브를 네트워크 마운트해서 사용했기 때문에 경로가 이와 같이 됩니다.
google_drive_base_path = 'google_drive/chemical/'

clear_output()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import cv2

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import numpy as np
import pandas as pd
import os
import os.path as pth
import time
from tqdm import tqdm

from rdkit import Chem
from rdkit import DataStructs
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')  

from IPython.display import clear_output

from multiprocessing import Process, Queue
import datetime
from IPython.display import clear_output

import gc

In [ ]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import SGD

import numpy as np
import tensorflow.keras.backend as K

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpus[0], True)

데이터를 구글 드라이브에서 wget을 통해 가져옵니다.  
이 방식을 많이 사용하면 구글 드라이브 다운로드 제한에 걸리지만, 마운트하는 것에 비해 빠르기에 사용하였습니다.

In [ ]:
import requests

from multiprocessing import Pool
from functools import partial

import zipfile
from google.colab import drive


def download_file_from_google_drive(id_, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()
    response = session.get(URL, params = { 'id' : id_ }, stream = True)
    token = get_confirm_token(response)
    if token:
        params = { 'id' : id_, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
        
    basename = response.headers['Content-Disposition'].split(';')[1].split('filename=')[1].replace('\"', '')
    # if 'Content-Disposition' not in response.headers:
    #     basename = id_
    # else:
        # basename = response.headers['Content-Disposition'].split(';')[1].split('filename=')[1].replace('\"', '')
    full_dst_filenname = pth.join(destination, basename)
    save_response_content(response, full_dst_filenname)
    return full_dst_filenname

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
file_id_list = [
    # '1nHqhEySQdWus7tiU5Bnd2vwfqHAiGxGo', # train.zip
    # '1Rw_HeEE6y57oosmWQsGPgk2T4jrykxm-', # train_half.zip
    # '1E6D8S8Sa5VcUi2MCOt_J5EQT41pS9C-6', # train_quat.zip
    # '1tEUNK9N5FlRt0tSNpyWIpZn5VSMPUCFh', # test.zip
    '1JvRvYMZQGCYXGcankPKiyzNvkCib0MXo', # train.csv
    '1JGruGKR-Tv1aKgONY0r_LW4zuwkLEyDX', # sample_submission.csv
]

In [ ]:
destination = 'data' ### YOUR_DOWNLOAD_PATH
if not pth.exists(destination):
    os.makedirs(destination, exist_ok=True)

    filename_list = []

    # ### Use single process
    # for file_id in file_id_list:
    #     filename = download_file_from_google_drive(id_=file_id, destination=destination)
    #     print('{} is done!'.format(filename))
    #     filename_list.append(filename)

    ### If you want to download more faster
    download_func = partial(download_file_from_google_drive, destination=destination)
    with Pool(4) as pool:
        for i, filename in tqdm(enumerate(pool.imap_unordered(download_func, file_id_list)), total=len(file_id_list)):
            print('{} is done!'.format(filename))
            filename_list.append(filename)

 50%|█████     | 1/2 [00:01<00:01,  1.06s/it]

data/sample_submission.csv is done!


100%|██████████| 2/2 [00:03<00:00,  1.51s/it]

data/train.csv is done!


### ChemHub에서 수집한 데이터를 이용해 학습

제공 된 데이터는 ChemHub에서 순서대로 수집한 SMILES로 유사 구조들이 밀집되어 있을 가능성이 높습니다.

In [ ]:
data_base_path = 'data'
os.makedirs(data_base_path, exist_ok=True)

In [ ]:
# test_dataset_name = 'test.tfrecords'
test_dataset_name = 'test.zip'

train_csv_name = 'train.csv'
sample_submission_name = 'sample_submission.csv'

In [ ]:
if not pth.exists(pth.join(data_base_path, test_dataset_name)):
    os.system('cp {}/data/{} {}'.format(google_drive_base_path, test_dataset_name, data_base_path))

In [ ]:
os.system('unzip {} -d {}'.format(pth.join(data_base_path, test_dataset_name), data_base_path))

0

In [ ]:
encoder_base_channel = 8
encoder_model_base = 'CustomDenseNet-121'
encoder_model_name = 'Autoencoder_{}_trts_basech_{:03d}'.format(encoder_model_base, encoder_base_channel)

In [ ]:
encoder_model_base_path = pth.join(data_base_path, 'checkpoint')
encoder_model_path = pth.join(encoder_model_base_path, encoder_model_name)
encoder_model_gdrive_path = pth.join(google_drive_base_path, 'model', 'checkpoint', encoder_model_name)

In [ ]:
os.makedirs(encoder_model_path, exist_ok=True)
target_checkpoint_filename = sorted(os.listdir(encoder_model_gdrive_path))[-1]
encoder_model_filename = pth.join(encoder_model_path, target_checkpoint_filename)
encoder_model_gdrive_filename = pth.join(encoder_model_gdrive_path, target_checkpoint_filename)

if not pth.exists(encoder_model_filename):
    os.system('cp {} {}'.format(encoder_model_gdrive_filename, encoder_model_filename))
    while os.path.getsize(encoder_model_gdrive_filename) != os.path.getsize(encoder_model_filename):
        os.system('cp {} {}'.format(encoder_model_gdrive_filename, encoder_model_filename))  

In [ ]:
n_mht = 512
# n_layer = 4
n_dff = 1024
n_head = 8
# dropout = 0.1
dropout = 0
decoder_model_name = 'trfrm_mht_{}_layer_{}_dff_{}_head_{}_DO_{}'.format(
    n_mht, n_layer, n_dff, n_head, dropout
)

In [ ]:
# model_name = 'enc-tr_{}_dec-tr_{}_len-100-all'.format(encoder_model_name, decoder_model_name)

최대 길이 및 워드 임베딩을 설정합니다

In [ ]:
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

# max_length = calc_max_length(train_captions)
max_length = 100

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=False, char_level=True)
all_token_list = [
    'c', 'C', '(', ')', '1', 'O', '=', '2', 'N', '<', '>', 'n', '[',
    ']', '3', '@', 'H', 'l', 'S', '-', 'F', '+', '4', 's', 'o', '#',
    'B', 'r', '.', '/', 'P', 'i', 'I', '5', '\\', 'e', 'A', 'a', 'g',
    '6', 'u', 't', 'T', 'M', 'b', 'K', 'Z', '8', 'd', '9', 'R', 'G',
    '7', 'L', 'V', 'h', 'W', 'p', 'm', 'E', 'Y', '0', 'U', 'f', 'D',
    'y', 'k', 'X', ' ', '^', '%', '$'
]
tokenizer.fit_on_texts(all_token_list)
top_k = len(tokenizer.word_index)

### 하이퍼 파라미터 및 학습에 필요한 변수 지정

In [ ]:
BATCH_SIZE = 100
BUFFER_SIZE = 100
d_model = n_mht
num_layers = n_layer
dff = n_dff
num_heads = n_head
vocab_size = top_k # + 1
dropout_rate = dropout

In [ ]:
is_sgd = True

EPOCHS = 200
learning_rate = 1e-4

### 모델 정의

In [ ]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates


def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)


def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)


def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights    

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])


class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)


    def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
        # enc_output.shape == (batch_size, input_seq_len, d_model)

        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(
            enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

        ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

        return out3, attn_weights_block1, attn_weights_block2 


class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                                 look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights


class CNN_Encoder(tf.keras.Model):
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        target_checkpoint_filename = sorted(os.listdir(encoder_model_path))[-1]
        image_autoencoder = load_model(pth.join(encoder_model_path, target_checkpoint_filename))
        image_features_extract_model = image_autoencoder.get_layer(encoder_model_base)
#         image_features_extract_model.trainable = False
        self.feature_extract_model = image_features_extract_model
        self.fc = tf.keras.layers.Dense(embedding_dim, activation='relu')
        
    def call(self, x):
        x = self.feature_extract_model(x)
        x = tf.keras.layers.Reshape((-1, x.shape[3]))(x)
        x = self.fc(x)
        return x


class ImageCaptioningTransformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff,
               target_vocab_size, pe_target, rate=0.1):
        super(ImageCaptioningTransformer, self).__init__()

        self.encoder = CNN_Encoder(d_model)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                               target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inp, tar, training, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp)

        # dec_output.shape == (batch_size, tar_seq_len, d_model)
        dec_output, attention_weights = self.decoder(
            tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

        return final_output, attention_weights


def create_masks(tar):
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return combined_mask        

### Optimizer 및 사용되는 함수

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=5):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
if is_sgd == True:
    optimizer = tf.keras.optimizers.SGD(lr=learning_rate)
else:
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)

# learning_rate = CustomSchedule(d_model)
# optimizer = tf.keras.optimizers.Adam(
#     learning_rate, beta_1=0.9, beta_2=0.98, 
#     epsilon=1e-9
# )

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none'
)

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [ ]:
def calculate_similarity(real, pred):
#     pred = np.array(list(map(np.array, pred)))
#     pred = np.moveaxis(pred, (0,1,2), (1,0,2))
    pred = np.argmax(pred, axis=-1)
#     print(real[:5], pred[:5])
    real = real.numpy()
    
    score_list = []
    for score_i, (each_pred, each_real) in enumerate(zip(pred, real)): 
        each_pred = ''.join([tokenizer.index_word.get(mol_i, '') for mol_i in each_pred])
        each_pred = each_pred.split('>')[0]
        m_pred = Chem.MolFromSmiles(each_pred)
        if m_pred == None:
            score_list.append(0)
            continue
        each_real = ''.join([tokenizer.index_word.get(mol_i, '') for mol_i in each_real])
        each_real = each_real[1:-1]
        m_real = Chem.MolFromSmiles(each_real)
        
        fp_pred = Chem.RDKFingerprint(m_pred)
        fp_real = Chem.RDKFingerprint(m_real)
        target_similarity = DataStructs.FingerprintSimilarity(fp_real,fp_pred)
        score_list.append(target_similarity)
        
    return score_list

# 인퍼런스 부분

### Predict 함수 정의

In [ ]:
# @tf.function
def predict(img_tensor):
    decoder_input = tf.expand_dims([tokenizer.word_index['<']] * img_tensor.shape[0], 1)
    output = decoder_input
    
    real_output = np.zeros((img_tensor.shape[0], max_length+1))
    real_output[...] = tokenizer.word_index[' ']
    real_output[:,0] = tokenizer.word_index['<']
    
    is_end_array = np.zeros((img_tensor.shape[0])).astype(np.bool)

    for i in range(max_length):
        if not all(is_end_array):
            combined_mask = create_masks(output)
            predictions, attention_weights = captioning_transformer(
                inp=img_tensor, tar=output, training=False, 
                look_ahead_mask=combined_mask, dec_padding_mask=None
            )
        else:
            break
        predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        real_output[:,i+1] = predicted_id.numpy().squeeze()
        output = tf.concat([output, predicted_id], axis=-1)
        is_end_array = (is_end_array | (real_output[:,i+1]==tokenizer.word_index['>']))
        # print(i, is_end_array)

    real_output[:,-1] = tokenizer.word_index['>']
#     return tf.squeeze(output, axis=0), attention_weights
    return real_output, attention_weights

# TestSet 예측

Transformer는 RNN과 다르게 initial random state가 없기 때문에 같은 모델에서 반복해서 예측해도 다른 결과를 얻을 수 없습니다.  
따라서 예측한 결과가 기본적인 smiles 분자식에 맞지 않을 경우엔, 다른 체크포인트에서 예측한 결과로 메꾸거나, 다른 모델에서 예측한 결과를 통해 항상 유효한 smiles식을 생성할 수 있도록 합니다.


In [ ]:
TEST_PATH = pth.join(data_base_path, 'test')

with open(pth.join(data_base_path, 'sample_submission.csv'), 'r') as csv_file:
    data = csv_file.read()
    
test_img_path = []

for line in data.split('\n')[1:-1]:
    image_id, _ = line.split(',')
    full_image_path = pth.join(TEST_PATH, image_id)

    test_img_path.append(full_image_path)
  
test_tfrecord_list = np.array([img_name+'.tfrecords' for img_name in test_img_path])
test_tfrecord_list[:3]

array(['data/test/test_0.png.tfrecords', 'data/test/test_1.png.tfrecords',
       'data/test/test_2.png.tfrecords'], dtype='<U34')

In [ ]:
image_feature_description_test = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    # 'filename': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function_test(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description_test)

def map_func_pred(target_record):
    img = target_record['image_raw']
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.dtypes.cast(img, tf.float32)
    return img

def prep_func_pred(image):
    result_image = image
    return result_image

### 결과 예측

In [ ]:
model_1_name = 'enc-tr_Autoencoder_CustomDenseNet-121_trts_basech_008_dec-tr_trfrm_mht_512_layer_4_dff_1024_head_8_DO_0_len-100-all'
model_2_name = 'enc-tr_Autoencoder_CustomDenseNet-121_trts_basech_008_dec-tr_trfrm_mht_512_layer_8_dff_1024_head_8_DO_0_len-100-all'

checkpoint_base_path = pth.join(google_drive_base_path, 'model', 'checkpoint')
checkpoint_path_list = [
    pth.join(model_1_name, 'ckpt-155'),
    pth.join(model_2_name, 'ckpt-120'),
    pth.join(model_1_name, 'ckpt-141'),
    pth.join(model_1_name, 'ckpt-147'),
    pth.join(model_2_name, 'ckpt-111'),
    pth.join(model_1_name, 'ckpt-149'),
    pth.join(model_2_name, 'ckpt-121'),
    pth.join(model_1_name, 'ckpt-151'),
    pth.join(model_1_name, 'ckpt-144'),
    pth.join(model_2_name, 'ckpt-109'),
    pth.join(model_1_name, 'ckpt-139'),
    pth.join(model_1_name, 'ckpt-138'),
    pth.join(model_1_name, 'ckpt-137'),
]

구글 드라이브에서 바로 체크포인트를 불러오면 I/O 명령이 너무 많은 탓인지 종종 disk quota limit 에러가 뜹니다.  
그래서 저는 코랩 로컬에 옮겨와서 모델을 불러오지만, 실행하실 때는 첨부한 체크포인트를 data/checkpoint 이하 경로에 놓고 실행해주시면 됩니다.


In [ ]:
# ### 해당 코드는 구글 드라이브에서 코랩 로컬로 필요한 체크포인트를 옮기는 코드입니다.
# for target_checkpoint_path in tqdm(checkpoint_path_list):
#     model_name = pth.dirname(target_checkpoint_path)
#     temp_checkpoint_path = pth.join('data', 'checkpoint', model_name)
#     os.makedirs(temp_checkpoint_path, exist_ok=True)

#     checkpoint_basename = pth.basename(target_checkpoint_path)
#     temp_checkpoint_filename = pth.join(temp_checkpoint_path, checkpoint_basename)
#     origin_checkpoint_filename = pth.join(checkpoint_base_path, target_checkpoint_path)

#     if not pth.exists(temp_checkpoint_filename+'.data-00000-of-00001'):
#         os.system('cp {}* {}'.format(origin_checkpoint_filename, temp_checkpoint_path))
#         while os.path.getsize(temp_checkpoint_filename+'.data-00000-of-00001') != os.path.getsize(origin_checkpoint_filename+'.data-00000-of-00001'):
#             os.system('cp {}* {}'.format(origin_checkpoint_filename, temp_checkpoint_path))  



  0%|          | 0/13 [00:00<?, ?it/s]

 77%|███████▋  | 10/13 [00:16<00:04,  1.62s/it]

 85%|████████▍ | 11/13 [00:25<00:07,  3.78s/it]

 92%|█████████▏| 12/13 [00:31<00:04,  4.61s/it]

100%|██████████| 13/13 [00:40<00:00,  3.09s/it]


In [ ]:
result_submission_path = pth.join('submission')
os.makedirs(result_submission_path, exist_ok=True)

submission_df = pd.read_csv(pth.join(data_base_path, 'sample_submission.csv'))

pred_summary_array = np.zeros((len(submission_df), max_length)).astype(np.int32)
pred_summary_array[...] = tokenizer.word_index[' ']
unavaliable_mask = np.ones(len(submission_df)).astype(np.bool)

for target_checkpoint_path in checkpoint_path_list:
    model_name = pth.dirname(target_checkpoint_path)
    temp_checkpoint_path = pth.join('data', 'checkpoint', model_name)
    os.makedirs(temp_checkpoint_path, exist_ok=True)

    checkpoint_basename = pth.basename(target_checkpoint_path)
    temp_checkpoint_filename = pth.join(temp_checkpoint_path, checkpoint_basename)
    origin_checkpoint_filename = pth.join(checkpoint_base_path, target_checkpoint_path)

    ### GPU 메모리 관리를 위해 모델이 이미 존재하면 지워줍니다.
    if 'captioning_transformer' in locals():
        K.clear_session()
        del(captioning_transformer)
        gc.collect()

    target_model_layer = int(model_name.split('_layer_')[1].split('_dff_')[0])
    captioning_transformer = ImageCaptioningTransformer(
        target_model_layer, d_model, num_heads, dff,
        vocab_size, pe_target=max_length,
        rate=dropout_rate
    )

    ckpt = tf.train.Checkpoint(
        captioning_transformer=captioning_transformer, 
        optimizer=optimizer
    )
    ckpt_manager = tf.train.CheckpointManager(ckpt, temp_checkpoint_filename, max_to_keep=25)
    ckpt.restore(temp_checkpoint_filename)

    dataset_test = tf.data.TFRecordDataset(test_tfrecord_list[unavaliable_mask], compression_type='GZIP')
    dataset_test = dataset_test.map(_parse_image_function_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # dataset_test = dataset_test.cache()
    dataset_test = dataset_test.map(map_func_pred, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset_test = dataset_test.batch(BATCH_SIZE)
    dataset_test = dataset_test.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    test_num_steps = int(np.ceil(unavaliable_mask.sum()/BATCH_SIZE))
    # test_num_steps = len(test_img_path)
    test_result = []
    for img_tensor in tqdm(dataset_test, total=test_num_steps, position=0, leave=True):
        pred_result, attention_weights = predict(img_tensor)
        test_result.append(pred_result)
    test_result = np.concatenate(test_result)

    preds = []
    for rid in range(len(test_result)):
        pred = ''.join([tokenizer.index_word[i] for i in test_result[rid]])
        pred = pred[1:].split('>')[0]
        preds.append(pred)

    mask_backup = unavaliable_mask.copy()
    temp_array = pred_summary_array[mask_backup].copy()
    temp_mask = np.ones(unavaliable_mask.sum()).astype(np.bool)
    error_idx = []
    for i, pred in enumerate(preds):
        m = Chem.MolFromSmiles(pred)
        if m == None:
            error_idx.append(i)
        else:
            temp_array[i] = test_result[i][:max_length]
            temp_mask[i] = False
    pred_summary_array[mask_backup] = temp_array
    unavaliable_mask[unavaliable_mask] = temp_mask

    error_idx = np.array(error_idx)
    error_idx_ = error_idx.copy()
    print('Model name:', temp_checkpoint_filename)
    print('Error count:', len(error_idx))

preds = []
for each_pred in pred_summary_array.astype(np.int32):
    pred = ''.join([tokenizer.index_word.get(i, '') for i in each_pred])
    pred = pred[1:].split('>')[0]
    preds.append(pred)
preds = np.array(preds)
preds[unavaliable_mask] = 'CCC(C)Oc1cc(N)nc(S(=O)(=O)c2cccc(C)c2)n1' ### 끝까지 예측하지 못한 이미지는 임의의 값을 집어넣습니다.
submission_df['SMILES'] = preds

### submission file 저장
result_submission_path = 'submission'
os.makedirs(result_submission_path, exist_ok=True)

today_str = datetime.date.today().strftime('%Y%m%d')
result_filename = 'model_1_2_ensemble.csv'
submission_csv_fileaname = pth.join(result_submission_path, '_'.join([today_str, result_filename]))
submission_df.to_csv(submission_csv_fileaname, index=False)

100%|██████████| 200/200 [31:50<00:00,  9.55s/it]


Model name: data/checkpoint/enc-tr_Autoencoder_CustomDenseNet-121_trts_basech_008_dec-tr_trfrm_mht_512_layer_4_dff_1024_head_8_DO_0_len-100-all/ckpt-155
Error count: 46


100%|██████████| 1/1 [00:14<00:00, 14.55s/it]


Model name: data/checkpoint/enc-tr_Autoencoder_CustomDenseNet-121_trts_basech_008_dec-tr_trfrm_mht_512_layer_8_dff_1024_head_8_DO_0_len-100-all/ckpt-120
Error count: 9


100%|██████████| 1/1 [00:09<00:00,  9.28s/it]


...

Model name: data/checkpoint/enc-tr_Autoencoder_CustomDenseNet-121_trts_basech_008_dec-tr_trfrm_mht_512_layer_4_dff_1024_head_8_DO_0_len-100-all/ckpt-139
Error count: 1


100%|██████████| 1/1 [00:08<00:00,  8.66s/it]


Model name: data/checkpoint/enc-tr_Autoencoder_CustomDenseNet-121_trts_basech_008_dec-tr_trfrm_mht_512_layer_4_dff_1024_head_8_DO_0_len-100-all/ckpt-138
Error count: 1


100%|██████████| 1/1 [00:08<00:00,  8.05s/it]


Model name: data/checkpoint/enc-tr_Autoencoder_CustomDenseNet-121_trts_basech_008_dec-tr_trfrm_mht_512_layer_4_dff_1024_head_8_DO_0_len-100-all/ckpt-137
Error count: 1


In [ ]:
!head submission/20201012_model_1_2_ensemble.csv

file_name,SMILES
test_0.png,COc1cc(N2CCOCC2)ccc1C=C1C(=O)NC(=O)N(C2CCCCC2)C1=O
test_1.png,CC(=NN=C1C(=O)Nc2cc(Cl)ccc21)c1cc2ccccc2oc1=O
test_2.png,CC1CN(Cc2ccccc2NC(=O)NCC(C)(C)C(N)=O)CC(C)O1
test_3.png,Cc1cccc2c(NCCc3nnc4ccccn34)c(C#N)cnc12
test_4.png,COc1cccc(-c2cc(CNC(=O)C(c3ccc(F)cc3)N(C)C)no2)c1
test_5.png,Cc1c(Cl)cccc1NC(=O)C1CCN(S(C)(=O)=O)c2cc(Cl)ccc2O1
test_6.png,Cc1ccc(C(=O)N2CCc3ccccc32)cc1N
test_7.png,CCC(Nc1cc(C(=O)N2CCCC2)ccn1)c1ccc(C)cn1
test_8.png,Cn1c(=NC(=O)c2ccccc2O)[nH]c2cc(Cl)ccc21
